# Project on Decision Trees

In [41]:
import numpy as np
import math

## Question 1

In [42]:
def dataFromFile(f):
    dataFile = open(f,'r')
    lines = dataFile.readlines()
    records = []
    attributes = [[],[],[]]
    types = lines[0].split(',')
    mot = types[-1].strip('\n')
    types[-1] = mot
    types = np.array(types)
    for line in lines[1:]:
        current = []
        cutLine = line.split(',')
        attributes[0].append(int(cutLine[0]))
        attributes[1].append(int(cutLine[1]))
        attributes[2].append(int(cutLine[2]))
        for i in cutLine:
            current.append(int(i))
        records.append(current)
    for i in range(0,3):
        dicattribute = dict.fromkeys(attributes[i])
        attributes[i] = list(dicattribute)
        attributes[i] = sorted(attributes[i])
    records = np.array(records)
    dataFile.close()
    return records, attributes, types

In [43]:
records,attributes,types = dataFromFile('/Users/Tiffany/Desktop/telecom/2A/SD201/TP3/LabDecisionTreeSD201/data/data.csv')
print(records, attributes,types)

[[1 3 2 0]
 [0 1 0 1]
 [0 3 2 1]
 ...
 [0 3 2 0]
 [1 1 0 1]
 [1 3 1 0]] [[0, 1], [1, 2, 3], [0, 1, 2]] ['Sex' 'Pclass' 'Embarked' 'Survived']


### GINI

In [44]:
def gini(group):
    total = len(group)
    C1=sum(group)
    C2=total - C1
    g = 1 - (C1/total)**2 - (C2/total)**2
    return g

In [45]:
def gini2(D):
    group = []
    for line in D:
        group.append(line[3])
    return gini(group)

In [46]:
def splitData(records,columnNb,splitValue):
    recordSplit = [[],[]]
    for line in records:
        if line[columnNb]<splitValue:
            recordSplit[0].append(line)
        else:
            recordSplit[1].append(line)
    return recordSplit

In [47]:
def giniSplit(records, attributes, columnNb):
    dataToSplit = [[],[]]
    possibleValues = attributes[columnNb]
    for line in records:
        dataToSplit[0].append(line[columnNb])
        dataToSplit[1].append(line[3]) #survived or not: 1 or 0
    if len(possibleValues)>2:
        giniSplitList = []
        for i in range(len(possibleValues)-1):
            res1 = []
            res2 = []
            for j in range(len(dataToSplit[0])):
                if  dataToSplit[0][j] <= possibleValues[i]:
                    res1.append(dataToSplit[1][j])
                else:
                    res2.append(dataToSplit[1][j])
            gini1 = gini(res1)
            gini2 = gini(res2)
            giniSp = (gini1*len(res1)/(len(res2)+len(res1)))+(gini2*len(res2)/(len(res2)+len(res1)))
            giniSplitList.append(giniSp)
        indexValue = giniSplitList.index(min(giniSplitList))
        splitValue = possibleValues[indexValue]
        recordSplit = splitData(records,columnNb,splitValue)
    else:
        splitValue = possibleValues[0]
        recordSplit = splitData(records,columnNb,splitValue)
    return recordSplit, splitValue

### DECISION TREE CONSTRUCTION

In [48]:
#From a file, we import data and launch the recursive function
def BuildDecisionTree(f, minNum):
    D,  A, names= dataFromFile(f)
    return Build(D,A, minNum,0, names)

In [56]:
#Recursive fonction to create the DecisionTree
def Build(D, A, minNum, v, names):
    classBool = D[0][3]
    print("ok")
    sameBool = True
    s = 0
    DGini = []
    for i in D:
        if i[3] != classBool:
            sameBool = False
        s += i[3]
    if sameBool == True : #if the all surviv
        return [0, classBool, D]
    elif len(D) < minNum or v>2:
        g = gini2(D)
        if s < len(D) - s : #on a plus de 0 que de 1
            return [g, 0, D]
        else : 
            return [g, 1, D]
    else : 
        g = gini2(D)
        recordSplit, splitValue = giniSplit(D,A,v)
        D1 = recordSplit[0]
        D2 = recordSplit[1]
        return [g, names[v] +'<='+ str(splitValue), Build(D1, A, minNum, v+1,names), Build(D2, A, minNum, v+1,names)]

In [57]:
BuildDecisionTree('/Users/Tiffany/Desktop/telecom/2A/SD201/TP3/LabDecisionTreeSD201/data/data.csv',20)

ok


IndexError: list index out of range

In [30]:
#Test
print(dataFromFile('/Users/Tiffany/Desktop/telecom/2A/SD201/TP3/LabDecisionTreeSD201/data/data.csv'))
dT = BuildDecisionTree('/Users/Tiffany/Desktop/telecom/2A/SD201/TP3/LabDecisionTreeSD201/data/data.csv',20)
print(dT)

(array([[1, 3, 2, 0],
       [0, 1, 0, 1],
       [0, 3, 2, 1],
       ...,
       [0, 3, 2, 0],
       [1, 1, 0, 1],
       [1, 3, 1, 0]]), [[0, 1], [1, 2, 3], [0, 1, 2]], array(['Sex', 'Pclass', 'Embarked', 'Survived'], dtype='<U8'))


IndexError: list index out of range

In [218]:
#Question 1b

def linTree(decisionTree):
    T = []
    for i in range(2**4 -1):
        T.append([])
    lin(decisionTree, 0, T)
    return T

In [219]:
def lin(tree, position, T):
    T[position].append(tree[0])
    T[position].append(tree[1])
    if tree[1] == 0 or tree[1] == 1 : #it is a leaf
        T[position].append(tree[2])
    else:
        lin(tree[2], 2*position+1, T)
        lin(tree[3], 2*position+2, T)

In [220]:
def printDecisionTree(decisionTree):
    decisionTree = linTree(decisionTree)
    S = ''
    depth = int(math.log2(len(decisionTree)+1)) #graph depth
    for i in range(depth):
        length = 2**i
        for j in range (length): #We browse every node every levels
            position = 2**i -1 +j
            if decisionTree[position]!= [] :
        #We label the type of node and add its level and feature
                if (position==0):
                    S+= 'Root \n'
                    S+= 'Level ' + str(i) + '\n'
                    S+= 'Feature ' + str(decisionTree[position][1]) +'\n'
                elif (2*position >= len(decisionTree)-1):
                    S+= 'Leaf \n'
                    S+= 'Level ' + str(i) + '\n'
                    S+= 'Class ' + str(decisionTree[position][1]) + '\n'
                elif (decisionTree[2*position +1]==[]):
                    S+= 'Leaf \n'
                    S+= 'Level ' + str(i) + '\n'
                    S+= 'Class ' + str(decisionTree[position][1]) + '\n'
                else : 
                    S+= 'Intermediate'
                    S+= '\n Level ' + str(i) + '\n'
                    S+= 'Feature ' + str(decisionTree[position][1]) +'\n'
          
                S+= 'Gini '+ str(decisionTree[position][0]) +'\n' 

                condition = False #determines if we haven't finish this level
                for k in range (j+1,length):
                    if decisionTree[2**i-1+k]!= []:
                        condition = True
                if (condition): #If we haven't
                    S+= '*****\n'
                else :
                    S += '\n'
    print (S)

In [224]:
#Test 
printDecisionTree(dT)

Root 
Level 0
Feature Sex<=0
Gini 0.4723650263627057

Intermediate
 Level 1
Feature Pclass<=2
Gini 0.384430473372781
*****
Intermediate
 Level 1
Feature Pclass<=1
Gini 0.3064437162277842

Intermediate
 Level 2
Feature Embarked<=1
Gini 0.10140306122448985
*****
Intermediate
 Level 2
Feature Embarked<=1
Gini 0.5
*****
Intermediate
 Level 2
Feature Embarked<=0
Gini 0.46560064498790643
*****
Intermediate
 Level 2
Feature Embarked<=0
Gini 0.24174858108924058

Leaf 
Level 3
Class 1
Gini 0.03702385190459241
*****
Leaf 
Level 3
Class 1
Gini 0.12945179584120986
*****
Leaf 
Level 3
Class 1
Gini 0.4228316326530613
*****
Leaf 
Level 3
Class 0
Gini 0.46875
*****
Leaf 
Level 3
Class 0
Gini 0.48185941043083896
*****
Leaf 
Level 3
Class 0
Gini 0.455
*****
Leaf 
Level 3
Class 0
Gini 0.3503025987896047
*****
Leaf 
Level 3
Class 0
Gini 0.22524194945669662




In [ ]:
def pruneTree(decisionTree,alpha,Tree):
    